In [1]:
from qat.interop.qiskit import QPUToBackend
from qat.qpus import PyLinalg,get_default_qpu
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute, Aer
from qat.comm.shared.ttypes import Batch
from qiskit.utils import QuantumInstance
from qiskit.opflow import CircuitSampler,StateFn,I,Z,One,CircuitStateFn

### Testing myqlm-to-qiskit module with simple circuit, Bell state

In [2]:
nbqubits = 2

qreg = QuantumRegister(nbqubits)
creg = ClassicalRegister(nbqubits)

qiskit_circuit = QuantumCircuit(qreg, creg)

qiskit_circuit.h(qreg[0])
qiskit_circuit.cx(qreg[0], qreg[1])
qiskit_circuit.measure(qreg, creg)


#ALTERNATICE CIRCUIT, LIKE THE ABOVE BUT WITHOUT MEASURE AND CREG
nbqubits = 3
qreg = QuantumRegister(nbqubits)
qiskit_circuit_2 = QuantumCircuit(qreg)

qiskit_circuit_2.h(qreg[0])
qiskit_circuit_2.cx(qreg[0], qreg[1])

In [3]:
shots = 100#000

#DEFINE 2 BACKENDS TO USE
qpu = PyLinalg()
qpu = get_default_qpu()
backendQLM = QPUToBackend(qpu)
print(backendQLM)

backendQiskit = Aer.get_backend('aer_simulator')

QiskitConnector


In [4]:
#PURE QLM MACHINE


result = execute(qiskit_circuit, backendQLM, shots=shots).result()

print(result.results)


[ExperimentResult(shots=100, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3': 39, '0x0': 61}), header=QobjExperimentHeader(memory_slots=2, metadata=None))]


In [5]:
#PURE QLM MACHINE THROUGH QUANTUM INSTANCE (VERY SIMILAR TO THE ONE ABOVE)
quantum_instance = QuantumInstance(backend=backendQLM, shots=shots, max_credits=None)
result = quantum_instance.execute(qiskit_circuit) # had_transpiled=True
print(result.results)



[ExperimentResult(shots=100, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 57, '0x3': 43}), header=QobjExperimentHeader(memory_slots=2, metadata=None))]


In [6]:
#QISKIT BACKEND

quantum_instance = QuantumInstance(backend=backendQiskit, shots=shots, max_credits=None)
result = quantum_instance.execute(qiskit_circuit) # had_transpiled=True
print(result.results)


[ExperimentResult(shots=100, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3': 50, '0x0': 50}), header=QobjExperimentHeader(clbit_labels=[['c0', 0], ['c0', 1]], creg_sizes=[['c0', 2]], global_phase=0.0, memory_slots=2, metadata=None, n_qubits=2, name='circuit-7', qreg_sizes=[['q0', 2]], qubit_labels=[['q0', 0], ['q0', 1]]), status=DONE, seed_simulator=2690986483, metadata={'parallel_state_update': 8, 'noise': 'ideal', 'measure_sampling': True, 'device': 'CPU', 'num_qubits': 2, 'parallel_shots': 1, 'remapped_qubits': False, 'method': 'stabilizer', 'active_input_qubits': [0, 1], 'num_clbits': 2, 'input_qubit_map': [[1, 1], [0, 0]], 'fusion': {'enabled': False}}, time_taken=0.00040588200000000004)]


In [7]:
#QISKIT BACKEND WITH CIRCUIT SAMPLER INSTEAD
quantum_instance = QuantumInstance(backend=backendQiskit,shots=shots,max_credits=None)
sampler = CircuitSampler(quantum_instance)


exp = ~StateFn(Z@Z@Z) @ CircuitStateFn(qiskit_circuit_2)
converted = sampler.convert(exp)
print(converted)
converted.eval()



ComposedOp([
  OperatorMeasurement(ZII),
  DictStateFn({'011': 0.7416198487095663, '000': 0.6708203932499369})
])


(1+0j)

In [8]:
#QLM BACKEND WITH CIRCUIT SAMPLER INSTEAD

quantum_instance = QuantumInstance(backend=backendQLM,shots=shots,max_credits=None)
sampler = CircuitSampler(quantum_instance)

exp = ~StateFn(Z@Z) @ CircuitStateFn(qiskit_circuit_2)
converted = sampler.convert(exp)
print(converted)
converted.eval()

ComposedOp([
  OperatorMeasurement(III),
  DictStateFn({'110': 0.6633249580710799, '000': 0.7483314773547883})
])


(1+0j)

BUG FIXED: {'memory_slots': 3} 
Problem encounterd in qiskit-result.py line ~296
Fixed in providers.py from myqlm-interop
This metadata missing in the header of the result from MyQLM causes the conversion from hexa to binary to miss the extra 0 in the highest bits, e.g. 00011100 would be 11100, so it misrepresent the qubits present